# Gas Station Optimization Problem
#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

Learn how to modularize a simulation so that we can seach over different inputs to find an optimum.

## Overview

Now that you have completed Alice's simulation, she want you to answer her over all question: How many pumps should she have at her gas station. She wants to know how many pumps are needed to maximize gallons sold.  In order to do this we will impove on the code from the Gas Station Simulation Problem. Feel free to use either your own code or the code in the solution as a starting point. 

## The details

### Encapsulate the Simulation

We need to rewrite the simulation as a encapsulated function that takes every varible the simulation needs to run as an argument. This will require that for every function or object, the parameters are passed explicitly as arguments; rather than being read as global varialbes.

This will allow us to easily see the effect of gas pumps on the number of gallons sold.

Tips:
* We can either eliminate the print statement of include a flag telling whether or not the simulation should print results. The simulation and optimization will run much faster if they don't have to print and the output will be much cleaner.
* Since we are only focused on the number of gallons sold, we don't need the other metrics. Since the GasStation instance keeps track of all of the gas sold, we don't need to save the Car instances in a list.

### Simulate for Different Numbers of Pumps

Now since we have a function that returns the simulated number of gallons sold, we can to run this function for a number of values to see its results.

### Plot The Reusulting Correspondence


# Gas Station Optimization Solution

## Setup

In [13]:
import simpy

import numpy as np
import pandas as pd

from scipy.stats import expon
from scipy.stats import norm

In [2]:
class GasStation():
    
    """
    Models a gas station as a simpy.Resource with a given number of pumps.
    
    Arguments:
    -----------
    num_pumps (int): The number of pumps at the gas station
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, num_pumps, env):
        
        self.num_pumps = num_pumps
        
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=num_pumps)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_station (GasStation): The gas station the cars are going to
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_station,
        pump_rate,
        env,
        verbose
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_station = gas_station
        self.pump_rate = pump_rate
        self.env = env
        
        self.verbose = verbose
        
        self.wait_time = np.inf
        self.finished = False
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        if self.verbose:
            print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
                
        with self.gas_station.resource.request() as req:
            
            yield req
            
            pump_start_time = self.env.now
            
            if self.verbose:
                print(f'Car {self.id} begins utilizing a pump at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / self.pump_rate
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            if self.verbose:
                print(f'Car {self.id} leaves its pump at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            self.gas_station.minutes_utilized += utilization
            self.gas_station.gallons_sold += self.gas_required

In [4]:
def scheduler(
    expected_wait,
    gas_required_mean,
    gas_required_std,
    pump_rate,
    expected_lay_time,
    gas_station,
    cars,
    env,
    verbose
):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=expected_wait)
        
        std_norm = norm.rvs()
        gas_required = std_norm * gas_required_std + gas_required_mean
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=expected_lay_time)
        
        yield env.timeout(waiting_time)
        
        cars.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_station=gas_station,
                pump_rate = pump_rate,
                env=env,
                verbose=verbose
            )
        )
        
        id += 1

In [6]:
def sim(
    num_pumps,
    expected_wait,
    gas_required_mean,
    gas_required_std,
    pump_rate,
    expected_lay_time,
    sim_time,
    verbose = False
):
    
    env = simpy.Environment()
    cars = []
    gas_station = GasStation(num_pumps, env)
    env.process(
        scheduler(
            expected_wait,
            gas_required_mean,
            gas_required_std,
            pump_rate,
            expected_lay_time,
            gas_station,
            cars,
            env,
            verbose=verbose
        )
    )
    env.run(until=sim_time)
    
    total_gallons_sold = gas_station.gallons_sold
    
    return total_gallons_sold

## Create Fixed Parameters

In [5]:
fixed_parameters = {
    'expected_wait': 2,
    'gas_required_mean': 15,
    'gas_required_std': 5,
    'pump_rate': 10,
    'expected_lay_time': 5,
    'sim_time': 24 * 60,
}

## Do a test run of the simulation

In [12]:
gas_sold = sim(
    num_pumps=1,
    **fixed_parameters,
    verbose=True
)

gas_sold

Car 0 arrives at 0.56 minutes
Car 0 begins utilizing a pump at 0.56 minutes
Car 1 arrives at 1.48 minutes
Car 2 arrives at 4.75 minutes
Car 3 arrives at 4.93 minutes
Car 4 arrives at 11.63 minutes
Car 5 arrives at 13.97 minutes
Car 6 arrives at 14.8 minutes
Car 0 leaves its pump at 15.1 minutes
Car 1 begins utilizing a pump at 15.1 minutes
Car 7 arrives at 15.8 minutes
Car 8 arrives at 17.38 minutes
Car 9 arrives at 17.95 minutes
Car 10 arrives at 18.48 minutes
Car 11 arrives at 21.7 minutes
Car 1 leaves its pump at 22.92 minutes
Car 2 begins utilizing a pump at 22.92 minutes
Car 12 arrives at 23.08 minutes
Car 13 arrives at 23.33 minutes
Car 14 arrives at 23.77 minutes
Car 2 leaves its pump at 25.03 minutes
Car 3 begins utilizing a pump at 25.03 minutes
Car 15 arrives at 25.3 minutes
Car 16 arrives at 25.57 minutes
Car 17 arrives at 26.87 minutes
Car 3 leaves its pump at 27.46 minutes
Car 4 begins utilizing a pump at 27.46 minutes
Car 4 leaves its pump at 29.21 minutes
Car 5 begins ut

Car 605 arrives at 1273.26 minutes
Car 606 arrives at 1275.25 minutes
Car 225 leaves its pump at 1276.03 minutes
Car 226 begins utilizing a pump at 1276.03 minutes
Car 607 arrives at 1276.31 minutes
Car 608 arrives at 1282.68 minutes
Car 609 arrives at 1283.44 minutes
Car 226 leaves its pump at 1283.81 minutes
Car 227 begins utilizing a pump at 1283.81 minutes
Car 610 arrives at 1284.63 minutes
Car 227 leaves its pump at 1284.94 minutes
Car 228 begins utilizing a pump at 1284.94 minutes
Car 228 leaves its pump at 1285.61 minutes
Car 229 begins utilizing a pump at 1285.61 minutes
Car 611 arrives at 1287.03 minutes
Car 612 arrives at 1287.25 minutes
Car 229 leaves its pump at 1287.3 minutes
Car 230 begins utilizing a pump at 1287.3 minutes
Car 613 arrives at 1288.7 minutes
Car 614 arrives at 1292.47 minutes
Car 615 arrives at 1292.51 minutes
Car 616 arrives at 1293.04 minutes
Car 230 leaves its pump at 1294.09 minutes
Car 231 begins utilizing a pump at 1294.09 minutes
Car 617 arrives at 

3941.342676299086

## Do the optimization

In [26]:
opt_table = pd.Series(
    {num_pumps: sim(num_pumps, **fixed_parameters) for num_pumps in range(1,100)}
)

In [27]:
arg_max = opt_table.idxmax()
max_gallons_sold = opt_table[arg_max]

print(f'The maximum number of gallons sold is {max_gallons_sold} with {arg_max} pumps')

The maximum number of gallons sold is 11836.786805017862 with 20 pumps


In [28]:
opt_table

1      4080.965944
2      8492.600814
3     10694.272836
4     11052.666176
5     10479.229251
6     10799.362991
7     10831.053711
8     10071.038441
9     10573.109999
10    11009.322899
11    10703.233626
12    10633.932052
13    10791.337663
14    10085.465211
15    11078.750537
16    10273.254086
17    10880.515655
18    11005.558327
19    10628.064028
20    11836.786805
21    10337.573771
22    10791.903022
23    10791.205182
24    10627.054493
25    10429.552900
26    10783.533985
27    10443.913804
28    10305.721474
29    11119.834344
30    10932.595168
          ...     
70    10559.154434
71    10937.038262
72    10242.759374
73    11355.032517
74    10471.542620
75    10425.894185
76    10671.274387
77    10859.360676
78    11378.201494
79    11032.834143
80    10756.276766
81    10765.831583
82    10383.358881
83    11262.497329
84    10417.325827
85    10856.729358
86    10219.270067
87    10323.663524
88    10552.109862
89    10526.434651
90    10977.847495
91    10891.